## Effect size de las asociaciones: columna `OR_or_beta`

El dataset de GWAS Catalog mezcla dos tipos de valor diferentes en una sola columna: el OR (Odds Ratio) de una asociación en caso de fenotipos binarios, y el $\beta$ de asociación en caso de fenotipos continuos. Estos valores no son equiparables, pues no están en la misma escala. El OR es un factor *multiplicativo*: indica, por cada alelo de riesgo, por cuánto *se multiplica* el riesgo de tener la enfermedad. Por ende, el $OR = 1$ es el valor de ausencia de efecto y, en el dataset, esperamos ver valores $OR > 1$ reportados (los $OR \approx 1$, por definición, no corresponden a una asociación significativa). Por otro lado, los $\beta$ provienen de una regresión lineal y corresponden a un efecto *aditivo*: indican por cada alelo cuánto se suma al valor del fenotipo (por ejemplo, cuánto crece en media el Body Mass Index al añadir un alelo de cierta variante). Por ende, $\beta = 0$ es el valor de ausencia de efecto y, en el dataset, esperamos ver valores de $\beta > 0$.

Por otro lado, si bien es posible tener $\beta < 0$ (efecto negativo en el fenotipo) o $0 < OR < 1$ (efecto "protector" del alelo, disminuye el riesgo), no vemos esto en el dataset porque lo reportado es siempre el alelo de "strongest risk".

Por ende, separaremos los ORs de los $\beta$ con una regla sencilla:

  * Si el valor está en $[0, 1]$, asumimos que es un $\beta$. Esos fenotipos son marcados como "continuos".
  * Marcamos como $\beta$ otros valores $> 1$ que estén en variantes para esos mismos fenotipos continuos.
  * El resto serán considerados Odds Ratios.

Otros supuestos influyen en esta decisión. Tal vez el más importante es el conocimiento previo de que la inmensa mayoría de las variantes genéticas que afectan fenotipos complejos (*complex traits*) tienen efectos pequeños.

NOTAS:

 * Confirmar este criterio mirando un fenotipo que sepamos es binario. ¿Alguna evidencia de que es válido (más de allá mirar un fenotipo)?
 *  Mirar en GWAS catalog si hay alguna descripción.

In [3]:
%run imports.py

data = pd.read_table(f"{results_dir}/gwas_cat.filtrado.tsv.gz")
data

,fenotipo,alelo_riesgo,categoria_fenotipo,genes_reportados,genes_mapeados,contexto,OR_or_beta,EFO_URI
0,Crohn's disease,rs12722489-C,Digestive system disorder,IL2RA,IL2RA,intron,1.1100,http://www.ebi.ac.uk/efo/EFO_0000384
1,Crohn's disease,rs151181-G,Digestive system disorder,IL27 | SH2B1 | EIF3C | LAT | CD19,CLN3 | AC138894.1,intron,1.0700,http://www.ebi.ac.uk/efo/EFO_0000384
2,Crohn's disease,rs2058660-G,Digestive system disorder,IL12RL2 | IL18R1 | IL1RL1 | IL18RAP,IL18RAP,intron,1.1900,http://www.ebi.ac.uk/efo/EFO_0000384
3,Crohn's disease,rs6556412-A,Digestive system disorder,IL12B,AC008691.1,intron,1.1800,http://www.ebi.ac.uk/efo/EFO_0000384
4,Crohn's disease,rs2797685-A,Digestive system disorder,VAMP3,PER3 | Z98884.1,intron,1.0500,http://www.ebi.ac.uk/efo/EFO_0000384
...,...,...,...,...,...,...,...,...
112619,Highest math class taken (MTAG),rs902627-A,Biological process,NaN,DPYSL4,intron,0.0095,http://www.ebi.ac.uk/efo/EFO_0004875
112620,Lung adenocarcinoma,rs3117582-C,Cancer,APOM | BAT3,APOM,intron,1.2200,http://www.ebi.ac.uk/efo/EFO_0000571
112621,Lung adenocarcinoma,rs1051730-T,Cancer,CHRNA5 | CHRNA3,CHRNA3,synonymous,1.3100,http://www.ebi.ac.uk/efo/EFO_0000571
112622,Lung adenocarcinoma,rs31489-C,Cancer,CLPTM1L,CLPTM1L,intron,1.1200,http://www.ebi.ac.uk/efo/EFO_0000571


In [5]:
def deducir_si_fenotipo_es_binario(df):
    return any(df.OR_or_beta < 1)

fenos_binario_status = data.groupby("fenotipo").apply(deducir_si_fenotipo_es_binario)
data.fenotipo.map(fenos_binario_status)

0         False
1         False
2         False
3         False
4         False
          ...  
112619     True
112620    False
112621    False
112622    False
112623    False
Name: fenotipo, Length: 112624, dtype: bool